In [82]:
from IPython.display import HTML, clear_output,Javascript, display
from ipywidgets import widgets
from ipywidgets import Button, GridBox, Layout, ButtonStyle, Box, VBox
import requests
import json
import pandas as pd
from pprintpp import pprint as pp


# I wish i knew how to always hide this cell !
Javascript("Jupyter.notebook.execute_cells([2])")
HTML('''
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')



# Welcome


##### How to generate an API token
##### API documentation
 - CANVAS API DOC
 - Requests Library

#### Quick Directions
1. enter in api token
2. select canvas environment 
3. click submit ( make sure the top turns green )
4. 


In [83]:
error = widgets.Output()

API_widget = widgets.Text(placeholder='API TOKEN',
                       layout=Layout(width='90%',align_items='center', grid_area='main'))


Authentication = Button(
    value=False,
    description='NOT Authorized',
    disabled=True,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='unlock',
    layout=Layout(width='auto', grid_area='header')
)


Submit = Button(
    value=False,
    description='Submit',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='',
    layout=Layout(width='auto', grid_area='footer')
)



# for the env // Domain
environ = widgets.Select(
    options=['Production', 'Test', 'Beta'],
    value='Production',
    description='Environment:',
    disabled=False,
    rows=3,
    layout=Layout(width='auto', grid_area='sidebar')
)

environs = {'Production':'https://canvas.upenn.edu/api/v1/', 'Test':'b', 'Beta':'c'}



def set_up_forms():
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+10)'))

def authorize():
    # check api key 
    global headers
    headers = {
        'Authorization': 'Bearer %s' % (API_widget.value) ,
    }
    global domain
    domain = environs[environ.value]
    
    x = test_api() # returns status code of test response
    
    if API_widget.value and x ==200:
    # if good:
        

        # UI changes
        Authentication.icon = 'lock'
        Authentication.description = 'Authorized'
        Authentication.button_style ='success'
        set_up_forms()
    #Authentication.style.button_color = 'lightgreen'
    # if bad:
    else:
        error.append_stdout("Invalid API token")
        Authentication.description='NOT Authorized'
        Authentication.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
        Authentication.icon='unlock'
    # output note 
    # 


def on_button_clicked(b):
    # clear any output ( error output from authorize )
    error.clear_output()
    authorize()
    
    # clear_output(wait=True) 
    
Submit.on_click(on_button_clicked)


#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#The-Grid-layout

header  = Authentication
main    = API_widget
sidebar = environ
footer  = Submit



form = GridBox(children=[header, main, sidebar,footer],
        layout=Layout(
            width='80%',
            margin="2% 0% 2% 10%",
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            #grid_gap='5px 5px',
            grid_template_areas='''
            " . header header ."
            "main main sidebar sidebar"
            " . footer footer . "
            ''')
       )

display(form)
display(error)



def on_value_change(change):
    Authentication.description='NOT Authorized'
    Authentication.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
    Authentication.icon='unlock'
    headers = {
    'Authorization': 'Bearer %s' % (change['new']) ,
    }
    

API_widget.observe(on_value_change, names='value')



environs = {'Production':'https://canvas.upenn.edu/api/v1/', 'Test':'b', 'Beta':'c'}
domain = environs[environ.value]


def test_api():
    
    #headers = {
    #'Authorization': 'Bearer %s' % (API_widget.value) ,
    #}
    
    url = domain + 'accounts/self/scopes'
    response = requests.get(url,headers=headers)
    r = response.json()
    #print(response.status_code)
    return response.status_code




GridBox(children=(Button(button_style='warning', description='NOT Authorized', disabled=True, icon='unlock', l…

Output()

## Explore Endpoints
##### Description
... coming soon ...
list of all of the api endpoints and their associated info


In [84]:
#GET /api/v1/accounts/:account_id/scopes



#print(headers)
#print(domain)


ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

url = domain +'accounts/self/scopes'
response = requests.get(url,headers=headers)
r = response.json()
df_scopes = pd.DataFrame.from_dict(r, orient='columns')

output_df = widgets.Output()
dropdown_resource = widgets.Dropdown(description='Resource',
                                     options = unique_sorted_values_plus_ALL(df_scopes.resource_name),
                                    layout=Layout(width='40%',padding="5px 20px 20px 20px"))
dropdown_apimethod = widgets.Dropdown(description='API Method',
                                     options = unique_sorted_values_plus_ALL(df_scopes.verb),
                                    layout=Layout(width='40%',padding="5px 20px 20px 20px"))



def dropdown_resource_eventhandler(change):
    output_df.clear_output(wait=True)
    with output_df:
        if (change.new == ALL):
            display(df_scopes)
        else:
            display(df_scopes[df_scopes.resource_name == change.new])




def load_output():
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+1)'))


#pp(r)

dropdown_resource.observe(dropdown_resource_eventhandler, names='value')



display(dropdown_resource)


#load_output()

Dropdown(description='Resource', layout=Layout(padding='5px 20px 20px 20px', width='40%'), options=('ALL', 'AP…

In [ ]:
from IPython.core.display import HTML

display(HTML("<h3 style='padding-left:40px;'>Output:</h3>"))
display(output_df)

## Page Views 
#### Description 
[see canvas doc](https://canvas.instructure.com/doc/api/users.html#method.page_views.index)


#### Directions
- you can leave the fields blank or specify a date ( EST ) 
- the out put will be in utc so use the quick utc to est converter 
- I dont recommend pagination ...

In [85]:
# /api/v1/accounts/:account_id/scopes

df_pageview = pd.DataFrame.from_dict({}, orient='columns')

pageview_output = widgets.Output()
userinfo_output = widgets.Output()

user_id = widgets.Text(placeholder='user_id',
                       layout=Layout(width='90%',align_items='center', grid_area='header'))

start = widgets.DatePicker(
    description='start date',
    disabled=False,
    layout=Layout(width='90%',align_items='center', grid_area='left'))

end = widgets.DatePicker(
    description='end date',
    disabled=False,
    layout=Layout(width='90%',align_items='center', grid_area='right'))


Submit = Button(
    value=False,
    description='Get PageViews',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='',
    layout=Layout(width='auto', grid_area='footer')
)


header = user_id
left = start
right = end
footer = Submit

form = GridBox(children=[header, left, right,footer],
        layout=Layout(
            width='80%',
            margin="2% 0% 2% 10%",
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            #grid_gap='5px 5px',
            grid_template_areas='''
            " . header header ."
            "left left right right"
            " . footer footer . "
            ''')
       )

display(form)


#def validate_user(user_id):
#    # get the user's name so we can double check we have the correct person
#    user_id 

    
#def UTC_to_EST(time):
    # bad 
    

def page_views(user_id):
    # GET /api/v1/users/:user_id/page_views
    url = domain + 'users/self/page_views'
    params = {'per_page':'25'}
    response = requests.get(url,headers=headers,params=params)

    return response
    
    
def user_info(user_id):
    #url:GET|/api/v1/users/:id
    url = domain + 'users/'+user_id
    response = requests.get(url,headers=headers)
    print(url)
    return(response.json())
    #pp(response.json())
    
def on_button_clicked(b):
    # clear any output ( error output from authorize )
    pageview_output.clear_output(wait=True)
    userinfo_output.clear_output(wait=True)
    # Validate
    user = user_id.value
    userinfo = user_info(user)
    print("user",user)
    
    response = page_views(user)
    print(response.status_code)
    print(response.links)
    
    
    
    # format 
    r = response.json()
    
    # links is nested 
    #pp(response.json())
    df = pd.DataFrame.from_dict(r, orient='columns')
    
    print(userinfo)
    with userinfo_output:
        if userinfo:
            display('<h4> Page Results for' + userinfo['name'] + '(' +'id' +')</h4>')
        else:
             display('<h4> Page Results for <i>NONE</i></h4>')       
    with pageview_output:
        display(df)
    # clear_output(wait=True) 
    

    
Submit.on_click(on_button_clicked)


# should have pagination somewhere 

GridBox(children=(Text(value='', layout=Layout(align_items='center', grid_area='header', width='90%'), placeho…

In [ ]:
display(HTML("<h3 style='padding-left:40px;'>Output (PageViews):</h3>"))
display(userinfo_output)
display(pageview_output)

In [ ]:
import pandas as pd
from pprintpp import pprint as pp

url = domain +'accounts/self/scopes'
response = requests.get(url,headers=headers)
r = response.json()
#print(r)
#ok = pd.read_json(response)
#ok
#/api/v1/courses/:course_id/quizzes/:quiz_id/submission
#https://canvas.upenn.edu/courses/1424853/quizzes/2118685
user = '5507597'
course_id = '1424853'
quiz_id ='2118685'

url = '%s/courses/%s/quizzes/%s/submissions' % (domain,course_id,quiz_id)

params= {
    "per_page": 100,
}
#"include[user]": user 

response = requests.get(url,headers=headers,params=params)
print(response.url)


In [ ]:
#response.links
display("ok")

###### To Do 
- finish page views
- download link 
- feedback
###### SOURCES
 - https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html
 - https://canvas.instructure.com/doc/api/file.oauth.html#using-access-tokens
 - https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916
 
 


<hr>  

[Upenn Canvas Documentation](http://infocanvas.upenn.edu)  

Inline-style: 

<!-- ![ok](https://www.giving.library.upenn.edu/sites/all/themes/pennlibraries/images/penn_libraries_logo.png)
-->
[Upenn Library]   
[Maddy Hodges](https://mfhodges.github.io/)